In [7]:
import numpy as np
from sympy.physics.quantum.cg import CG

# Routines to compute multi-shell $SU(3)$ reduced matrix elements

In [17]:
# Function to load reduced SU(3) Wigner coefficients < (λμ)KL; (λ'μ')K'L'|| (λ''μ'')K''L'' > from external file
def loadWignerSU3(filename):

    
# ================================================================================================================ 
# Function to load SU(3) reduced matrix elements < (λ'μ') S' ||| a†{(η0)1/2} ||| (λμ) S > or 
#                                                < (λ'μ') S' ||| ã{(0η)1/2} ||| (λμ) S > 
#                                                from external file
def loadSU3rme(filename):

    
# ================================================================================================================
# Function to get a particular SU(3) Wigner coefficient
def getWignerSU3(η1, l1, η2, l2, lam0, mu0, K0, L0):
    
    
# ================================================================================================================   
# Function to get a particular SU(3) reduced matrix element
def getSU3rme(type_a, lam1, mu1, S1, lam0, mu0, S0, lam2, mu2, S2):
    
    
# ================================================================================================================
# Function to compute matrix element of coupled multi-shell one body operator
# < (λ'μ') K' L' M'_L' S' M'_S'| {a†ã}{(λ0 μ0) K0 L0 M_L0 S0 M_S0} | (λμ) K L M_L S M_S >
def oneBodyCoupledme(lam1, mu1, K1, L1, M_L1, S1, M_S1,
                      η1, l1, η2, l2,
                      lam0, mu0, K0, L0, M_L0, S0, M_S0,
                      lam2, mu2, K2, L2, M_L2, S2, M_S2):
    s = 0
    
    for ml1 in range(-l1, l2+1, 1):
        for ml2 in range(-l2, l2+1, 1):
            for ms1 in range(-1, 2, 2):
                for ms2 in range(-1, 2, 2):
                    
                    coeff = uncouplingCoeff(η1, l1, ml1, ms1/2, η2, l2, ml2, ms2/2, 
                                            lam0, mu0, K0, L0, M_L0, S0, M_S0)
                    
                    onebodyme = oneBodyUncoupledme(lam1, mu1, K1, L1, M_L1, S1, M_S1,
                                                   η1, l1, η2, l2,
                                                   lam0, mu0, K0, L0, M_L0, S0, M_S0,
                                                   lam2, mu2, K2, L2, M_L2, S2, M_S2)
                    
                    s += coeff*onebodyme
    
    return s

# ================================================================================================================
# Function to compute uncoupling coefficient of coupled {a†ã}{(λ0 μ0) K0 L0 M_L0 S0 M_S0} terms
def uncouplingCoeff(η1, l1, ml1, ms1, η2, l2, ml2, ms2, lam0, mu0, K0, L0, M_L0, S0, M_S0):
    
    SU3Coeff = getWignerSU3(η1, l1, η2, l2, lam0, mu0, K0, L0)
    SO3Coeff = float(CG(j1=l1, m1=ml1, j2=l2, m2=ml2, j3=L0, m3=M_L0).doit())
    SU2Coeff = float(CG(j1=1/2, m1=ms1, j2=1/2, m2=ms2, j3=S0, m3=M_S0).doit())
    
    return SU3Coeff*SO3Coeff*SU2Coeff
    
# ================================================================================================================
# Function to compute multi-shell uncoupled one-body matrix element
# < (λ'μ') K' L' M'_L' S' M'_S'| a†{(η0)l m_l 1/2 m_s} ã{(0η)l m_l 1/2 m_s} | (λμ) K L M_L S M_S >
def oneBodyUncoupledme(lam1, mu1, K1, L1, M_L1, S1, M_S1,
                       η1, l1, η2, l2,
                       lam0, mu0, K0, L0, M_L0, S0, M_S0,
                       lam2, mu2, K2, L2, M_L2, S2, M_S2):
    
    s = 0
    
    for lampp, mupp, Kpp, Lpp, M_Lpp, Spp, M_Spp in su3irreps:
        
        ad = ame(type_a, lam1, mu1, K1, L1, M_L1, S1, M_S1,
              lam0, mu0, K0, L0, M_L0, S0, M_S0,
              lampp, mupp, Kpp, Lpp, M_Lpp, Spp, M_Spp)
        
        at = ame(type_a, lampp, mupp, Kpp, Lpp, M_Lpp, Spp, M_Spp,
              lam0, mu0, K0, L0, M_L0, S0, M_S0,
              lam2, mu2, K2, L2, M_L2, S2, M_S2)
        
        s += ad*at  
    
    return s
    
# ================================================================================================================
# Function to compute < (λ'μ') K' L' M'_L' S' M'_S'| a†{(η0)l m_l 1/2 m_s} | (λμ) K L M_L S M_S > 
#                  or < (λ'μ') K' L' M'_L' S' M'_S'| ã{(0η)l m_l 1/2 m_s} | (λμ) K L M_L S M_S >
#                  from reduced matrix element and coupling coefficients
def ame(type_a, lam1, mu1, K1, L1, M_L1, S1, M_S1,
        lam0, mu0, K0, L0, M_L0, S0, M_S0,
        lam2, mu2, K2, L2, M_L2, S2, M_S2):
    
    rme = getSU3rme(type_a, lam1, mu1, S1, lam0, mu0, S0, lam2, mu2, S2)
    SO3Coeff = float(CG(j1=L2, m1=M_L2, j2=L0, m2=M_L0, j3=L1, m3=M_L1).doit())
    SU2Coeff = float(CG(j1=L2, m1=M_L2, j2=L0, m2=M_L0, j3=L1, m3=M_L1).doit())
    EdmundsFactor = 1/(np.sqrt(2*L1+1)*np.sqrt(2*S1+1))
    
    return rme*SO3Coeff*SU2Coeff*EdmundsFactor